In [16]:
import dataclasses
from dataclasses import dataclass

import numpy as np
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.nn.functional
from typing import List, Optional

from swarm_nfomp.utils.math import PointArray2D, RectangleRegionArray, interpolate_1d_pytorch
from swarm_nfomp.utils.position2d import Position2D
from swarm_nfomp.utils.timer import Timer
from swarm_nfomp.utils.universal_factory import UniversalFactory

In [17]:
@dataclass
class MovingObstacle:
    y_line: np.ndarray
    width: float
    length: float
    velocity: float
    x_initial_position: float


@dataclass
class MovingObject:
    position: Position2D
    time: float

In [18]:
@dataclass
class StampedPointArray:
    times: np.ndarray
    points: PointArray2D

    def as_numpy(self):
        return np.concatenate([self.times[:, np.newaxis], self.points.as_numpy()], axis=1)


@dataclass
class Point2D:
    x: float
    y: float

    def as_numpy(self):
        return np.array([self.x, self.y])

@dataclass
class MovingObstacleArray:
    y_line: np.ndarray
    width: np.ndarray
    length: np.ndarray
    velocity: np.ndarray
    x_initial_position: np.ndarray

    def inside(self, positions: StampedPointArray) -> np.ndarray:
        min_x = self.min_x(positions.times)
        min_y = self.min_y(positions.times)
        max_x = self.max_x(positions.times)
        max_y = self.max_y(positions.times)
        result = min_x <= positions.points.x[:, None]
        result &= positions.points.x[:, None] <= max_x
        result &= min_y <= positions.points.y[:, None]
        result &= positions.points.y[:, None] <= max_y
        return np.any(result, axis=1)

    def min_x(self, times):
        x_positions = self.x_initial_position[np.newaxis, :] + self.velocity * times[:, np.newaxis]
        result = x_positions - self.length[np.newaxis, :] / 2
        return result

    def max_x(self, times):
        x_positions = self.x_initial_position[np.newaxis, :] + self.velocity * times[:, np.newaxis]
        result = x_positions + self.length[np.newaxis, :] / 2
        return result

    def min_y(self, times):
        result = self.y_line[np.newaxis, :] - self.width[np.newaxis, :] / 2
        return np.repeat(result, len(times), axis=0)

    def max_y(self, times):
        result = self.y_line[np.newaxis, :] + self.width[np.newaxis, :] / 2
        return np.repeat(result, len(times), axis=0)

    @classmethod
    def from_dict(cls, data):
        y_line = np.array(data['y_line'])
        width = np.array(data['width'])
        length = np.array(data['length'])
        velocity = np.array(data['velocity'])
        x_initial_position = np.array(data['x_initial_position'])
        return cls(y_line, width, length, velocity, x_initial_position)

    def __len__(self):
        return len(self.x_initial_position)


class HighwayCollisionDetector:
    def __init__(self, moving_obstacle_array: MovingObstacleArray,
                 outside_rectangle_region_array: RectangleRegionArray):
        self.moving_obstacle_array = moving_obstacle_array
        self.outside_rectangle_region_array = outside_rectangle_region_array

    def is_collision(self, points: StampedPointArray):
        return self.moving_obstacle_array.inside(points) | (
            ~self.outside_rectangle_region_array.inside(points.points))

    @classmethod
    def from_dict(cls, data):
        moving_obstacle_array = MovingObstacleArray.from_dict(data['moving_obstacle_array'])
        outside_rectangle_region_array = RectangleRegionArray.from_dict(data['outside_rectangle_region_array'])
        return cls(moving_obstacle_array, outside_rectangle_region_array)


In [19]:
highway_collision_detector_parameters = {
    'moving_obstacle_array': {
        'y_line': [1, 2, 3],
        'width': [0.5, 0.6, 0.7],
        'length': [0.8, 0.9, 1.0],
        'velocity': [1, 1.75, 2],
        'x_initial_position': [0, 0, 0],
    },
    "outside_rectangle_region_array": [[-2, 24, 0, 4]]
}
collision_detector = HighwayCollisionDetector.from_dict(highway_collision_detector_parameters)

In [20]:
import itertools


def flat_lists(list_to_flat):
    return list(itertools.chain.from_iterable(list_to_flat))


class HighwayCollisionDetectorVisualizer:
    def __init__(self):
        self._fig = go.Figure()

    def visualize(self, detector: HighwayCollisionDetector, points: StampedPointArray):
        scatter = self.get_region_scatter(detector.outside_rectangle_region_array)
        self._fig.add_trace(scatter)
        self.add_moving_obstacles_frames(detector, points)
        self.update_layout()
        self._fig.update_yaxes(scaleanchor="x", scaleratio=1)

    @staticmethod
    def get_region_scatter(region, **kwargs):
        x_lists = [[region.min_x[i], region.max_x[i], region.max_x[i], region.min_x[i], region.min_x[i], None] for i in
                   range(len(region))]
        y_lists = [[region.min_y[i], region.min_y[i], region.max_y[i], region.max_y[i], region.min_y[i], None] for i in
                   range(len(region))]
        x = flat_lists(x_lists)
        y = flat_lists(y_lists)
        return go.Scatter(x=x[:-1], y=y[:-1], mode="lines", name="Outside region", **kwargs)

    def update_layout(self):
        self._fig.update_layout(
            updatemenus=[
                dict(
                    type="buttons",
                    direction="left",
                    showactive=False,
                    x=0.1,
                    y=1.2,
                    buttons=list([
                        dict(
                            label="Play",
                            method="animate",
                            args=[None, {
                                "frame": {
                                    "duration": 50,
                                    "redraw": False
                                },
                                "fromcurrent": True,
                                "transition": {
                                    "duration": 0,
                                }
                            }]
                        ),
                    ]),
                )
            ]
        )

    def add_moving_obstacles_frames(self, detector: HighwayCollisionDetector, points):
        times = points.times
        min_x = detector.moving_obstacle_array.min_x(times)
        max_x = detector.moving_obstacle_array.max_x(times)
        min_y = detector.moving_obstacle_array.min_y(times)
        max_y = detector.moving_obstacle_array.max_y(times)
        frames = []
        is_collision = detector.is_collision(points)
        print(is_collision)
        for i in range(len(times)):
            x_lists = [[min_x[i, j], max_x[i, j], max_x[i, j], min_x[i, j], min_x[i, j], None] for j in
                       range(max_y.shape[1])]
            y_lists = [[min_y[i, j], min_y[i, j], max_y[i, j], max_y[i, j], min_y[i, j], None] for j in
                       range(max_y.shape[1])]
            x = flat_lists(x_lists)[:-1]
            y = flat_lists(y_lists)[:-1]
            x_free = [points.points.x[i]] if not ifs_collision[i] else []
            y_free = [points.points.y[i]] if not is_collision[i] else []
            x_collision = [points.points.x[i]] if is_collision[i] else []
            y_collision = [points.points.y[i]] if is_collision[i] else []
            if i == 0:
                self._fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="MovingObstacles", fill="toself"))
                if len(y_collision) > 0:
                    self._fig.add_trace(go.Scatter(x=x_collision, y=y_collision, mode="markers", name="collision",
                                                   marker=dict(color="red")))
                if len(y_free) > 0:
                    self._fig.add_trace(
                        go.Scatter(x=x_free, y=y_free, mode="markers", name="free", marker=dict(color="green")))
            traces = [
                self.get_region_scatter(detector.outside_rectangle_region_array),
                go.Scatter(x=x, y=y, mode="lines", name="MovingObstacles", fill="toself"),
            ]
            if len(y_collision) > 0:
                traces.append(go.Scatter(x=x_collision, y=y_collision, mode="markers", name="collision",
                                         marker=dict(color="red")), )
            if len(y_free) > 0:
                traces.append(go.Scatter(x=x_free, y=y_free, mode="markers", name="free", marker=dict(color="green")))
            frames.append(go.Frame(data=traces))
        self._fig.frames = frames

    def save(self, filename):
        self._fig.write_html(filename)

In [21]:
visualizer = HighwayCollisionDetectorVisualizer()
test_times = np.linspace(0, 10, 100)
test_points = np.array([0, 1]) + np.linspace(0, 1, 100)[:, None] * np.array([20, 3])
visualizer.visualize(collision_detector, StampedPointArray(test_times, PointArray2D.from_vec(test_points)))
visualizer.save("data/highway_collision_detector_visualizer.html")

[ True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False]


# Dynamic NFOMP planner

In [22]:
@dataclasses.dataclass
class ONFModelConfig:
    mean: float
    sigma: float
    input_dimension: int
    encoding_dimension: int
    hidden_dimensions: List[int]


class ONF(nn.Module):
    def __init__(self, parameters: ONFModelConfig):
        super().__init__()
        self.encoding_layer = nn.Linear(parameters.input_dimension, parameters.encoding_dimension)
        self.mlp1 = self.make_mlp(parameters.encoding_dimension, parameters.hidden_dimensions[:-1],
                                  parameters.hidden_dimensions[-1])
        self.mlp2 = self.make_mlp(parameters.encoding_dimension + parameters.hidden_dimensions[-1], [], 1)
        self._mean = parameters.mean
        self._sigma = parameters.sigma

    @classmethod
    def make_mlp(cls, input_dimension, hidden_dimensions, output_dimension):
        modules = []
        for dimension in hidden_dimensions:
            modules.append(nn.Linear(input_dimension, dimension))
            modules.append(nn.ReLU())
            input_dimension = dimension
        modules.append(nn.Linear(input_dimension, output_dimension))
        return nn.Sequential(*modules)

    def forward(self, x):
        x = (x - self._mean) / self._sigma
        x = self.encoding_layer(x)
        x = torch.sin(x)
        input_x = x
        x = self.mlp1(input_x)
        x = torch.cat([x, input_x], dim=1)
        x = self.mlp2(x)
        return x


In [76]:
@dataclasses.dataclass
class PathPlannerTask:
    start: Point2D
    goal: Point2D
    start_time: float
    maximal_velocity: float
    collision_detector: HighwayCollisionDetector


@dataclasses.dataclass
class ResultPath:
    stamped_positions: StampedPointArray


@dataclasses.dataclass
class PathOptimizedState:
    positions_: torch.Tensor
    times_: torch.Tensor
    velocity_constraint_multipliers: torch.Tensor
    start_position: torch.Tensor
    goal_position: torch.Tensor
    start_time: float

    @property
    def result_path(self) -> ResultPath:
        return ResultPath(
            StampedPointArray(times=self.times.cpu().detach().numpy(),
                              points=PointArray2D.from_vec(self.positions.cpu().detach().numpy()))
        )

    @property
    def positions(self) -> torch.Tensor:
        return torch.cat(
            [self.start_position[None], self.positions_, self.goal_position[None]], dim=0)

    @property
    def times(self) -> torch.Tensor:
        return torch.cat(
            [torch.tensor([self.start_time]), self.times_], dim=0)

    def reparametrize(self):
        old_times: torch.Tensor = self.times
        new_times = torch.linspace(old_times[0].item(), old_times[-1].item(), old_times.shape[0])
        self.positions_.data = interpolate_1d_pytorch(self.positions, old_times, new_times)[1:-1]
        self.times_.data = new_times[1:]

    @property
    def stamped_positions(self) -> torch.Tensor:
        return torch.cat([self.times[:, None], self.positions], dim=1)

    @property
    def optimized_parameters(self):
        return [self.positions_, self.times_]

@dataclasses.dataclass
class OptimizerImplConfig:
    lr: float
    beta1: float
    beta2: float


class OptimizerImpl:
    def __init__(self, parameters: OptimizerImplConfig):
        self._optimizer = None
        self._parameters = parameters

    def setup(self, model_parameters):
        self._optimizer = torch.optim.Adam(model_parameters, lr=self._parameters.lr,
                                           betas=(self._parameters.beta1, self._parameters.beta2))

    def zero_grad(self):
        self._optimizer.zero_grad()

    def step(self):
        self._optimizer.step()


@dataclasses.dataclass
class OptimizerWithLagrangeMultipliersConfig(OptimizerImplConfig):
    lagrange_multiplier_lr: float


class OptimizerWithLagrangeMultipliers(OptimizerImpl):
    def __init__(self, parameters: OptimizerWithLagrangeMultipliersConfig):
        super().__init__(parameters)
        self._parameters = parameters
        self._lagrange_multiplier_parameters = None

    # noinspection PyMethodOverriding
    def setup(self, model_parameters, lagrange_multiplier_parameters):
        self._optimizer = torch.optim.Adam(model_parameters, lr=self._parameters.lr,
                                           betas=(self._parameters.beta1, self._parameters.beta2))
        self._lagrange_multiplier_parameters = lagrange_multiplier_parameters

    def zero_grad(self):
        self._optimizer.zero_grad()
        for p in self._lagrange_multiplier_parameters:
            p.grad = None

    def step(self):
        self._optimizer.step()
        with torch.no_grad():
            for p in self._lagrange_multiplier_parameters:
                p.data += self._parameters.lagrange_multiplier_lr * p.grad
                p.data = torch.where(p.data > 0, p, torch.zeros_like(p))


class PathOptimizedStateInitializer:
    def __init__(self, planner_task: PathPlannerTask, path_state_count: int, device: str):
        self._planner_task = planner_task
        self._path_state_count = path_state_count
        self._device = device

    def init(self) -> PathOptimizedState:
        with torch.no_grad():
            positions = self._initialize_positions()
            return PathOptimizedState(
                positions_=positions[1:-1].clone().detach().requires_grad_(True),
                times_=self._initialize_times(positions),
                velocity_constraint_multipliers=torch.zeros(self._path_state_count + 1, requires_grad=True,
                                                            device=self._device),
                start_position=torch.tensor(self._planner_task.start.as_numpy(), requires_grad=False, device=self._device, dtype=torch.float32),
                goal_position=torch.tensor(self._planner_task.goal.as_numpy(), requires_grad=False, device=self._device, dtype=torch.float32),
                start_time=self._planner_task.start_time,
            )

    def _initialize_positions(self) -> torch.Tensor:
        trajectory_length = self._path_state_count + 2
        trajectory = torch.zeros(self._path_state_count + 2, 2, requires_grad=True, device=self._device, dtype=torch.float32)
        start_point: np.ndarray = self._planner_task.start.as_numpy()
        goal_point: np.ndarray = self._planner_task.goal.as_numpy()
        trajectory[:, 0] = torch.linspace(start_point[0], goal_point[0], trajectory_length)
        trajectory[:, 1] = torch.linspace(start_point[1], goal_point[1], trajectory_length)
        return trajectory

    def _initialize_times(self, positions) -> torch.Tensor:
        distances = torch.linalg.norm(positions[1:] - positions[:-1], dim=1)
        cumulative_distances = torch.cumsum(distances, dim=0)
        cumulative_distances = torch.cat([torch.zeros(1), cumulative_distances], dim=0)
        times = self._planner_task.start_time + cumulative_distances / self._planner_task.maximal_velocity
        times = times[1:].clone().detach().requires_grad_(True)
        return times


@dataclasses.dataclass
class PathLossBuilderConfig:
    regularization_weight: float
    collision_weight: float
    time_regularization_weight: float
    soft_maximal_velocity_constraint_weight: float


class PathLossBuilder:
    def __init__(self, planner_task: PathPlannerTask, parameters: PathLossBuilderConfig):
        self._parameters = parameters
        self._planner_task = planner_task

    def get_loss(self, collision_model: nn.Module, optimized_state: PathOptimizedState):
        loss = self._parameters.regularization_weight * self._distance_loss(optimized_state)
        loss = loss + self._parameters.collision_weight * self._collision_loss(collision_model, optimized_state)
        loss = loss + self._parameters.time_regularization_weight * self._time_regularization_loss(optimized_state)
        loss = loss + self._parameters.soft_maximal_velocity_constraint_weight * self._soft_maximal_velocity_constraint_loss(
            optimized_state)
        return loss

    @staticmethod
    def _distance_loss(optimized_state: PathOptimizedState):
        path = optimized_state.positions
        return torch.mean((path[1:] - path[:-1]) ** 2)

    def _collision_loss(self, collision_model: nn.Module, optimized_state: PathOptimizedState):
        positions: torch.Tensor = optimized_state.stamped_positions
        positions = self.get_intermediate_points(positions)
        positions = positions.reshape(-1, 3)
        return torch.mean(torch.nn.functional.softplus(collision_model(positions)))

    @staticmethod
    def get_intermediate_points(positions):
        t = torch.rand(positions.shape[0] - 1, 1)
        delta = positions[1:] - positions[:-1]
        return positions[1:] + t * delta

    @staticmethod
    def _time_regularization_loss(optimized_state: PathOptimizedState):
        times = optimized_state.times
        return torch.mean((times[1:] - times[:-1]) ** 2)

    def _soft_maximal_velocity_constraint_loss(self, optimized_state: PathOptimizedState):
        positions = optimized_state.positions
        times = optimized_state.times
        distances = torch.linalg.norm(positions[1:] - positions[:-1], dim=1)
        time_differences = times[1:] - times[:-1]
        delta = time_differences * self._planner_task.maximal_velocity - distances
        return torch.mean(delta ** 2 + optimized_state.velocity_constraint_multipliers * delta)


class GradPreconditioner:
    def __init__(self, device, velocity_hessian_weight: float):
        self._device = device
        self._velocity_hessian_weight = velocity_hessian_weight
        self._inverse_hessian = None
        self._time_inverse_hessian = None

    def precondition(self, optimized_state: PathOptimizedState):
        point_count = optimized_state.positions_.shape[0]
        if self._inverse_hessian is None:
            self._inverse_hessian = self._calculate_inv_hessian(point_count)
            self._time_inverse_hessian = self._calculate_inv_hessian(point_count + 1)
        optimized_state.positions_.grad = self._inverse_hessian @ optimized_state.positions_.grad
        optimized_state.times_.grad = self._time_inverse_hessian @ optimized_state.times_.grad

    def _calculate_inv_hessian(self, point_count):
        hessian = self._velocity_hessian_weight * self._calculate_velocity_hessian(point_count) + np.eye(point_count)
        inv_hessian = np.linalg.inv(hessian)
        return torch.tensor(inv_hessian.astype(np.float32), device=self._device)

    @staticmethod
    def _calculate_velocity_hessian(point_count):
        hessian = np.zeros((point_count, point_count), dtype=np.float32)
        for i in range(point_count):
            hessian[i, i] = 4
            if i > 0:
                hessian[i, i - 1] = -2
                hessian[i - 1, i] = -2
        return hessian


class PathOptimizer:
    def __init__(self, timer: Timer, optimizer: OptimizerWithLagrangeMultipliers, loss_builder: PathLossBuilder,
                 state_initializer: PathOptimizedStateInitializer, grad_preconditioner: GradPreconditioner):
        self._loss_builder = loss_builder
        self._optimizer = optimizer
        self._timer = timer
        self._grad_preconditioner = grad_preconditioner
        self._state_initializer = state_initializer
        self._optimized_state: Optional[PathOptimizedState] = None

    def setup(self):
        self._optimized_state = self._state_initializer.init()
        self._optimizer.setup(self._optimized_state.optimized_parameters, [self._optimized_state.velocity_constraint_multipliers])

    def step(self, collision_model):
        self._optimizer.zero_grad()
        loss = self._loss_builder.get_loss(collision_model, self._optimized_state)
        self._timer.tick("trajectory_backward")
        loss.backward()
        self._timer.tock("trajectory_backward")
        self._timer.tick("inv_hes_grad_multiplication")
        self._grad_preconditioner.precondition(self._optimized_state)
        self._timer.tock("inv_hes_grad_multiplication")
        self._timer.tick("trajectory_optimizer_step")
        self._optimizer.step()
        self._timer.tock("trajectory_optimizer_step")

    def reparametrize(self):
        with torch.no_grad():
            self._optimized_state.reparametrize()

    @property
    def result_path(self) -> ResultPath:
        return self._optimized_state.result_path


class CollisionModelPointSampler:
    def __init__(self, fine_random_offset: float, course_random_offset: float):
        self._fine_random_offset = fine_random_offset
        self._course_random_offset = course_random_offset

    def sample(self, path: ResultPath) -> StampedPointArray:
        points: np.ndarray = path.stamped_positions.points.as_numpy()
        times = path.stamped_positions.times
        course_points = points + np.random.randn(*points.shape) * self._course_random_offset
        fine_points = points + np.random.randn(*points.shape) * self._fine_random_offset
        points = np.concatenate([course_points, fine_points], axis=0)
        times = np.concatenate([times, times], axis=0)
        return StampedPointArray(times=times, points=PointArray2D.from_vec(points))


class CollisionModelFactory:
    def __init__(self, parameters: ONFModelConfig):
        self._parameters = parameters

    def make_collision_model(self):
        return ONF(self._parameters)


class CollisionNeuralFieldModelTrainer:
    def __init__(self, timer: Timer, planner_task: PathPlannerTask,
                 collision_model_point_sampler: CollisionModelPointSampler,
                 optimizer: OptimizerImpl, collision_model_factory: CollisionModelFactory,
                 device: str):
        self._timer = timer
        self._collision_model_point_sampler = collision_model_point_sampler
        self._optimizer = optimizer
        self._collision_model_factory = collision_model_factory
        self._planner_task = planner_task
        self._collision_model: Optional[nn.Module] = None
        self._device = device
        self._collision_loss_function = nn.BCEWithLogitsLoss()
        self._timer = timer

    def setup(self):
        self._collision_model = self._collision_model_factory.make_collision_model()
        self._optimizer.setup(self._collision_model.parameters())

    def learning_step(self, path: ResultPath):
        self._timer.tick("optimize_collision_model")
        points = self._collision_model_point_sampler.sample(path)
        self._collision_model.requires_grad_(True)
        self._optimizer.zero_grad()
        predicted_collision = self._calculate_predicted_collision(points)
        truth_collision = self._calculate_truth_collision(points)
        truth_collision = torch.tensor(truth_collision.astype(np.float32)[:, None], device=self._device)
        loss = self._collision_loss_function(predicted_collision, truth_collision)
        loss.backward()
        self._optimizer.step()
        self._collision_model.requires_grad_(False)
        self._timer.tock("optimize_collision_model")

    def _calculate_predicted_collision(self, points: StampedPointArray):
        return self._collision_model(torch.tensor(points.as_numpy().astype(np.float32), device=self._device))

    def _calculate_truth_collision(self, points):
        return self._planner_task.collision_detector.is_collision(points)

    @property
    def collision_model(self) -> nn.Module:
        return self._collision_model


class SwarmNeuralFieldOptimalPathPlanner:
    def __init__(self, planner_task: PathPlannerTask,
                 collision_neural_field_model_trainer: CollisionNeuralFieldModelTrainer, path_optimizer: PathOptimizer,
                 iterations: int, reparametrize_rate: int):
        self._reparametrize_rate = reparametrize_rate
        self._planner_task = planner_task
        self._path_optimizer = path_optimizer
        self._collision_neural_field_model_trainer = collision_neural_field_model_trainer
        self._iterations = iterations

    def plan(self) -> ResultPath:
        self._path_optimizer.setup()
        self._collision_neural_field_model_trainer.setup()
        for i in range(self._iterations):
            self.step()
            if self._reparametrize_rate != -1 and i % self._reparametrize_rate == 0:
                self._path_optimizer.reparametrize()
        self._path_optimizer.reparametrize()
        return self._path_optimizer.result_path

    def step(self):
        path: ResultPath = self._path_optimizer.result_path
        self._collision_neural_field_model_trainer.learning_step(path)
        collision_model = self._collision_neural_field_model_trainer.collision_model
        self._path_optimizer.step(collision_model)

In [97]:
highway_collision_detector_parameters = {
    'moving_obstacle_array': {
        'y_line': [1, 2, 3, 1.5, 2.5, 0.5],
        'width': [0.5, 0.6, 0.7, 0.7, 1.0, 1.0],
        'length': [0.8, 0.9, 1.0, 1.0, 2.0, 1.5],
        'velocity': [1, 1.75, 2, 2.5, 3, 0.5],
        'x_initial_position': [0, -3, 0, -2.5, 2, 7],
    },
    "outside_rectangle_region_array": [[-2, 24, 0, 4]]
}
collision_detector = HighwayCollisionDetector.from_dict(highway_collision_detector_parameters)

input_planner_task = PathPlannerTask(
    start=Point2D(0., 2.),
    goal=Point2D(20., 2.),
    collision_detector=collision_detector,
    start_time=0,
    maximal_velocity=3,
)

planner_parameters = {
    "collision_neural_field_model_trainer": {
        "collision_model_point_sampler": {
            "course_random_offset": 5.,
            "fine_random_offset": 1.,
        },
        "optimizer": {
            "lr": 5e-2,
            "beta1": 0.9,
            "beta2": 0.9
        },
        "collision_model_factory": {
            "mean": 0,
            "sigma": 3.,
            "input_dimension": 3,
            "encoding_dimension": 64,
            "hidden_dimensions": [64, 64]
        }
    },
    "path_optimizer": {
        "optimizer": {
            "lr": 1e-2,
            "beta1": 0.9,
            "beta2": 0.9,
            "lagrange_multiplier_lr": 0.1
        },
        "loss_builder": {
            "regularization_weight": 100.,
            "collision_weight": 1.,
            "time_regularization_weight": 10.,
            "soft_maximal_velocity_constraint_weight": 100,

        },
        "state_initializer": {
            "path_state_count": 100,
        },
        "grad_preconditioner": {
            "velocity_hessian_weight": 0.5,
        }
    },
    "iterations": 1500,
    "reparametrize_rate": 10,
}
global_timer = Timer()
device_parameter = "cpu"
path_planner = UniversalFactory().make(SwarmNeuralFieldOptimalPathPlanner, planner_parameters,
                                             planner_task=input_planner_task, timer=global_timer,
                                             device=device_parameter)
result_path: ResultPath = path_planner.plan()

In [98]:
visualizer = HighwayCollisionDetectorVisualizer()
visualizer.visualize(collision_detector, result_path.stamped_positions)
visualizer.save("data/dynamic_path_planner_result_visualizer.html")

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False]
